## Kitchen training - test

In [1]:
import gdown
from sklearn.model_selection import train_test_split
import os
import shutil
import re
import pandas as pd

OD_PATH = '/models/research/object_detection/'
MAIN_PATH = '/happy-walrus/models/version_2/'
IMG_PATH = MAIN_PATH + 'images/'

### Resources

- [Tensorflow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection)
- [Tensorflow Object Detection API tutorial](https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10)

### Download data

Store all images and annotations in a single folder, zip the folder and store it in Google Drive under `safety/data/images/version_X` (replace the `X`).

In [45]:
# Download images
file_id = "1gnv7-2znYDCZ0Clf47A5KrqhUigBl3Ei"
file_name = "version_2.zip"
gdown.download(url="https://drive.google.com/uc?id=" + file_id,
               output=MAIN_PATH + file_name,
               quiet=False)

# Unzip images
os.mkdir(IMG_PATH)
!unzip -q {MAIN_PATH}{file_name} -d {IMG_PATH}
files_unzipped = os.listdir(MAIN_PATH + 'images/')
print('Files unzipped:', len(files_unzipped))
print('XML files:', len([file for file in files_unzipped if re.match('.*\.xml', file)]))
print('JPG files:', len([file for file in files_unzipped if re.match('.*\.jpg', file)]))

Downloading...
From: https://drive.google.com/uc?id=1gnv7-2znYDCZ0Clf47A5KrqhUigBl3Ei
To: /happy-walrus/models/version_2/version_2.zip
577MB [00:12, 47.2MB/s] 


Files unzipped: 5444
XML files: 2632
JPG files: 2811


Randomly split data into train and test:
```
|-images
    |- train
    |- test
```
where the last level folders will contain both the images and annotations (.xml) files.

In [49]:
# Create train and test folders in images/
os.mkdir(IMG_PATH + 'train')
os.mkdir(IMG_PATH + 'test')

# Get list of all files that have annotations and split into train and test
jpg_files = [file[:-4] for file in os.listdir(IMG_PATH) if re.search('.*\.jpg$', file)]
files = [file[:-4] for file in os.listdir(IMG_PATH) if re.search('.*\.xml$', file) and file[:-4] in jpg_files]
files = [file for file in files if not re.search('open_oven', file)]
files_train, files_test = train_test_split(files, train_size=0.9, test_size=0.1, random_state=2014)

# Move images to train or test folders
errors = []
for file in files_train:
    try:
        shutil.move(IMG_PATH + file + '.xml', IMG_PATH + 'train/' + file + '.xml')
        shutil.move(IMG_PATH + file + '.jpg', IMG_PATH + 'train/' + file + '.jpg')
    except Exception as err:
        errors.append([file, err])

for file in files_test:
    try:
        shutil.move(IMG_PATH + file + '.xml', IMG_PATH + 'test/' + file + '.xml')
        shutil.move(IMG_PATH + file + '.jpg', IMG_PATH + 'test/' + file + '.jpg')
    except Exception as err:
        errors.append([file, err])

# Remove remaining images
files_rm = [file for file in os.listdir(IMG_PATH) if re.search('.*\.jpg|.*\.xml', file)]
for file in files_rm:
    os.remove(IMG_PATH + file)
        
# Print errors
print("Removed files:", len(files_rm))
print("Number of errors:", len(errors))
print(errors)

Removed files: 251
Number of errors: 0
[]


### Generate data

Generate training and test data to `./images/`

In [56]:
# Generate training and test data
!python {OD_PATH}xml_to_csv.py
print("")
!head {IMG_PATH}/train_labels.csv -n 5

# Print results
print("")
unique_classes = sorted(pd.read_csv(IMG_PATH + 'train_labels.csv')['class'].unique())
print("Unique classes:", unique_classes)
print("Train images:", pd.read_csv(IMG_PATH + 'train_labels.csv')['filename'].unique().shape[0])
print("Test images:", pd.read_csv(IMG_PATH + 'test_labels.csv')['filename'].unique().shape[0])
print("")

# Generate label-id mapping
class_mapping = []
for count, item in enumerate(unique_classes):
    class_mapping.append('item{')
    class_mapping.append('\tid: ' + str(count + 1))
    class_mapping.append('\tname: \'' + item + '\'')
    class_mapping.append('}')
with open(MAIN_PATH + 'training/labelmap.pbtxt', 'w') as f:
    f.write('\n'.join(class_mapping))
print("Label id mapping:")
print('\n'.join(class_mapping))

Successfully converted xml to csv.
Successfully converted xml to csv.

filename,width,height,class,xmin,ymin,xmax,ymax
orange_kitchen.EH.033.jpg,1000,1000,stove,676,452,874,660
orange_kitchen.EH.033.jpg,1000,1000,chair,538,604,800,981
orange_kitchen.EH.033.jpg,1000,1000,chair,255,533,482,882
orange_kitchen.EH.033.jpg,1000,1000,chair,385,504,555,772

Unique classes: ['chair', 'ladder', 'oven', 'sofa', 'stool', 'stove']
Train images: 2192
Test images: 241

Label id mapping:
item{
	id: 1
	name: 'chair'
}
item{
	id: 2
	name: 'ladder'
}
item{
	id: 3
	name: 'oven'
}
item{
	id: 4
	name: 'sofa'
}
item{
	id: 5
	name: 'stool'
}
item{
	id: 6
	name: 'stove'
}


Open `generate_tfrecord.py` and:
1. replace `import tensorflow` with `import tensorflow.compat.v1`.
2. Modify `class_test_to_int()` function with label-id mapping.

In [57]:
# Generate TF formatted data
!python generate_tfrecord.py --csv_input={IMG_PATH}train_labels.csv --image_dir={IMG_PATH}train --output_path=train.record
!python generate_tfrecord.py --csv_input={IMG_PATH}test_labels.csv --image_dir={IMG_PATH}test --output_path=test.record
print("")
!ls -lh | grep .*\.record$

/root/.local/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:802: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 10. 
  warnings.warn(str(msg))
Successfully created the TFRecords: /happy-walrus/models/version_2/train.record
Successfully created the TFRecords: /happy-walrus/models/version_2/test.record

-rw-r--r-- 1 root root  34M Nov 13 20:06 test.record
-rw-r--r-- 1 root root 446M Nov 13 20:06 train.record


### Training

Get model checkpoint (other models in Tensorflow object detection's [Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)).

In [68]:
!wget "http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz"
!tar xvzf faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

--2019-11-13 21:22:59--  http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.6.144, 2607:f8b0:4000:80f::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.6.144|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149119618 (142M) [application/x-tar]
Saving to: ‘faster_rcnn_inception_v2_coco_2018_01_28.tar.gz.1’

faster_rcnn_incepti 100%[===================>] 142.21M  65.8MB/s    in 2.2s    

2019-11-13 21:23:06 (65.8 MB/s) - ‘faster_rcnn_inception_v2_coco_2018_01_28.tar.gz.1’ saved [149119618/149119618]

faster_rcnn_inception_v2_coco_2018_01_28/
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.index
faster_rcnn_inception_v2_coco_2018_01_28/checkpoint
faster_rcnn_inception_v2_coco_2018_01_28/pipeline.config
faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.data-00000-of-00001
faster_rcnn_inception_v2_coco_2018_01_28

1. Copy config file from `./samples/configs/` to `./training/`
2. Adapt parameters in config file:
    - `num_classes`
    - `fine_tune_checkpoint`
    - train_input_reader: `input_path`, `label_map_path`
    - `num_examples`: images/test/
    - eval_input_reader: `input_path`, `label_map_path`
    - `data_augmentation_options`, see augmentations and arguments in [preprocessor module](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto).

In [72]:
!cp {OD_PATH}samples/configs/faster_rcnn_inception_v2_pets.config {MAIN_PATH}training/faster_rcnn_inception_v2_kitchens.config

Move train.py from `legacy/` to `.`

In [80]:
!cp {OD_PATH}legacy/train.py .

Train model
- Make sure to delete all files in trainin/ except labelmap.pbtxt and the config file.
- needs to be stopped manually)

In [70]:
!python train.py --logtostderr --train_dir={MAIN_PATH}training/ --pipeline_config_path={MAIN_PATH}training/faster_rcnn_inception_v2_kitchens.config

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.






Instructions for updating:
Use object_detection/model_main.py.
W1113 21:26:20.188523 139957644097344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/absl/app.py:250: main (from __main__) is deprecated and will be removed in a future version.
Instructions for updating:
Use object_detection/model_main.py.

W1113 21:26:20.188791 139957644097344 module_wrapper.py:139] From train.py:90: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W1113 21:26:20.189162 139957644097344 module_wrapper.py:139] From /models/research/object_detection/utils/config_util.py:102: The name tf.gfile.G


W1113 21:26:21.043663 139957644097344 module_wrapper.py:139] From /models/research/object_detection/core/preprocessor.py:2937: The name tf.image.resize_images is deprecated. Please use tf.image.resize instead.

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).
W1113 21:26:21.068610 139957644097344 deprecation.py:323] From /models/research/object_detection/core/batcher.py:101: batch (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W1113 21:26:21.073147 139957644097344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/training/

Instructions for updating:
Use keras.layers.flatten instead.
W1113 21:26:24.940399 139957644097344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/contrib/layers/python/layers/layers.py:1634: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I1113 21:26:24.943832 139957644097344 regularizers.py:98] Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
I1113 21:26:24.966415 139957644097344 regularizers.py:98] Scale of 0 disables regularizer.

W1113 21:26:25.292551 139957644097344 module_wrapper.py:139] From /models/research/object_detection/core/losses.py:177: The name tf.losses.huber_loss is deprecated. Please use tf.compat.v1.losses.huber_loss instead.


W1113 21:26:25.295541 139957644097344 module_wrapper.py:139] From /models/research/object_detection/core/losses.py:1

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
W1113 21:26:33.066828 139957644097344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/contrib/slim/python/slim/learning.py:742: Supervisor.__init__ (from tensorflow.python.training.supervisor) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
2019-11-13 21:26:34.145352: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-11-13 21:26:34.157997: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2100060000 Hz
2019-11-13 21:26:34.158737: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xeeba7d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-11-13 21:26:34.158795: I tensorflow/compiler/xla/service/service.cc:176]   StreamE

INFO:tensorflow:global step 33: loss = 0.7154 (1.481 sec/step)
I1113 21:27:45.134356 139957644097344 learning.py:507] global step 33: loss = 0.7154 (1.481 sec/step)
INFO:tensorflow:global step 34: loss = 2.4186 (1.721 sec/step)
I1113 21:27:46.856645 139957644097344 learning.py:507] global step 34: loss = 2.4186 (1.721 sec/step)
INFO:tensorflow:global step 35: loss = 1.6456 (1.101 sec/step)
I1113 21:27:47.959243 139957644097344 learning.py:507] global step 35: loss = 1.6456 (1.101 sec/step)
INFO:tensorflow:global step 36: loss = 1.2512 (1.381 sec/step)
I1113 21:27:49.341932 139957644097344 learning.py:507] global step 36: loss = 1.2512 (1.381 sec/step)
INFO:tensorflow:global step 37: loss = 2.1360 (1.573 sec/step)
I1113 21:27:50.915874 139957644097344 learning.py:507] global step 37: loss = 2.1360 (1.573 sec/step)
INFO:tensorflow:global step 38: loss = 1.7744 (1.407 sec/step)
I1113 21:27:52.324133 139957644097344 learning.py:507] global step 38: loss = 1.7744 (1.407 sec/step)
INFO:tenso

In command line, run `tensorboard --logdir=/walrus/models/version_2/training`

### Export model

In [13]:
!ls -l training/

total 567122
-rwxr-xr-x 1 root root       269 Nov  4 16:18 checkpoint
-rwxr-xr-x 1 root root       136 Nov  4 14:50 desktop.ini
-rwxr-xr-x 1 root root  27840397 Nov  4 16:26 events.out.tfevents.1572880714.74a0ad80ad34
-rwxr-xr-x 1 root root      3574 Nov  4 15:16 faster_rcnn_inception_v2_kitchens.config
-rwxr-xr-x 1 root root  10466489 Nov  4 15:18 graph.pbtxt
-rwxr-xr-x 1 root root        79 Nov  4 15:09 labelmap.pbtxt
-rwxr-xr-x 1 root root 103009376 Nov  4 15:58 model.ckpt-1295.data-00000-of-00001
-rwxr-xr-x 1 root root     25561 Nov  4 15:58 model.ckpt-1295.index
-rwxr-xr-x 1 root root   5441806 Nov  4 15:58 model.ckpt-1295.meta
-rwxr-xr-x 1 root root 103009376 Nov  4 16:08 model.ckpt-1604.data-00000-of-00001
-rwxr-xr-x 1 root root     25561 Nov  4 16:08 model.ckpt-1604.index
-rwxr-xr-x 1 root root   5441806 Nov  4 16:08 model.ckpt-1604.meta
-rwxr-xr-x 1 root root 103009376 Nov  4 16:18 model.ckpt-1907.data-00000-of-00001
-rwxr-xr-x 1 root root     25561 Nov  4 16:18 model.ckpt-190

In [16]:
!python {OD_PATH}export_inference_graph.py --input_type image_tensor --pipeline_config_path training/faster_rcnn_inception_v2_kitchens.config --trained_checkpoint_prefix training/model.ckpt-1295 --output_directory inference_graph

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W1104 16:28:48.156948 140301050382144 module_wrapper.py:139] From /models/research/object_detection/export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1104 16:28:48.168290 140301050382144 module_wrapper.py:139] From /models/research/object_detection/exporter.py:389: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W1104 16:28:48.169805 140301050382144 module_wrapper.py:139] From /models/research/object_detection/exporter.py:121: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W1104 16:28:48.25


W1104 16:28:53.766587 140301050382144 module_wrapper.py:139] From /models/research/object_detection/exporter.py:268: The name tf.add_to_collection is deprecated. Please use tf.compat.v1.add_to_collection instead.

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
W1104 16:28:53.766983 140301050382144 deprecation.py:323] From /models/research/object_detection/exporter.py:370: get_or_create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step

W1104 16:28:53.770792 140301050382144 module_wrapper.py:139] From /models/research/object_detection/exporter.py:402: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

Instructions for updating:
Use `tf.profiler.profile(graph, run_meta, op_log, cmd, options)`. Build `options` with `tf.profiler.ProfileOptionBuilder`. See README

234 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*,.*Initializer.*,.*Regularizer.*,.*BiasAdd.*
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================
Incomplete shape.

Doc:
scope: The nodes in the model graph are organiz


W1104 16:28:55.322115 140301050382144 module_wrapper.py:139] From /models/research/object_detection/exporter.py:419: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.


W1104 16:28:56.493039 140301050382144 module_wrapper.py:139] From /models/research/object_detection/exporter.py:332: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-11-04 16:28:56.493579: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2019-11-04 16:28:56.493635: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2019-11-04 16:28:56.493702: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (74a0ad80ad34): /proc/driver/nvidia/version does not exist
2019-11-04 16:28:56.494113: I tensorflow/core/platform


W1104 16:29:05.716187 140301050382144 module_wrapper.py:139] From /models/research/object_detection/utils/config_util.py:188: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.

INFO:tensorflow:Writing pipeline config file to inference_graph/pipeline.config
I1104 16:29:05.716538 140301050382144 config_util.py:190] Writing pipeline config file to inference_graph/pipeline.config


In [17]:
!ls -lh inference_graph/

total 101M
-rwxr-xr-x 1 root root   77 Nov  4 16:28 checkpoint
-rwxr-xr-x 1 root root  136 Nov  4 14:50 desktop.ini
-rwxr-xr-x 1 root root  50M Nov  4 16:29 frozen_inference_graph.pb
-rwxr-xr-x 1 root root  50M Nov  4 16:28 model.ckpt.data-00000-of-00001
-rwxr-xr-x 1 root root  16K Nov  4 16:28 model.ckpt.index
-rwxr-xr-x 1 root root 1.8M Nov  4 16:28 model.ckpt.meta
-rwxr-xr-x 1 root root 3.1K Nov  4 16:29 pipeline.config
drwxrwxrwx 2 root root    0 Nov  4 16:29 saved_model


### Perform inference

In [18]:
!pwd

/walrus/kitchens_test


To `Object_detection_image.py`:
1. add before last line `cv2.imwrite('/path/to/save/boxed_image.JPG', image)` and comment last line out.
2. `NUM_CLASSES`
2. `IMAGE_NAME`

In [20]:
!cat Object_detection_image.py

If `NoneType` related error, reinstall opencv with `apt install opencv-python`.

In [21]:
!python Object_detection_image.py

Traceback (most recent call last):
  File "Object_detection_image.py", line 29, in <module>
    from utils import label_map_util
ModuleNotFoundError: No module named 'utils'
